In [175]:
import config as cfg
import model as m
import dataset as d

import os
import pandas as pd


import re
import time
import random
import datetime

import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

sns.set()

In [176]:
train_df = pd.read_csv('/DATA/image-search/kgg/input/train.csv')
train_df.sentiment.value_counts()

neutral     11118
positive     8582
negative     7781
Name: sentiment, dtype: int64

In [177]:
11118/8582, 8582/7781, 7781/8582

(1.2955022139361454, 1.1029430664439017, 0.9066651130272664)

In [178]:
test_df = pd.read_csv('/DATA/image-search/kgg/input/test.csv')
test_df.sentiment.value_counts()

testID = set(test_df.textID.values)
testID = set()

In [192]:
test_df.sentiment.value_counts()

neutral     1430
positive    1103
negative    1001
Name: sentiment, dtype: int64

In [179]:
df = pd.read_csv('./dataset/pseudo_selected.csv')
df['wcnt'] = df.apply(lambda row: len(row.selected_text.strip().split()), axis=1)

In [180]:
cnt = 0
rm_indice = []
for idx, textID in enumerate(df.textID):
    if textID in testID:
        rm_indice.append(idx)
df = df.drop(rm_indice).reset_index(drop=True)
df

,textID,text,sentiment,selected_text,wcnt
0,0142108215,Thank you so much phaoloo !!!!,positive,Thank you,2
1,fb08563a7b,Midnight ice-cream weather! So **** bored,negative,bored,1
2,9a2c6ae21c,Ohh i forgot to tell you last night that when ...,positive,amazing,1
3,5de3c34293,i am the only arabic girl who`s online every ...,neutral,i am the only arabic girl who`s online every ...,14
4,cf0d831059,starting the video editing of the first spanki...,neutral,starting the video editing of the first spank...,22
...,...,...,...,...,...
3515,1e1cb8d9a4,let me dream about my spock now goodnight,neutral,let me dream about my spock now goodnight,8
3516,d65a8658b0,I just got through doing my work now I can tak...,positive,its great,2
3517,136b1926c9,Cool. That`d be fantastic!,positive,Cool. That`d be fantastic!,4
3518,242910ff98,'We never want to lose the fans that got us he...,positive,Thank you for saying that Wish you and the ot...,14


In [194]:
pos_df = df[df.sentiment == 'positive'].reset_index(drop=True)
# pos_df = pos_df[pos_df.wcnt > 1].reset_index(drop=True)
pos_df = pos_df.drop_duplicates(['selected_text'], keep='first').reset_index(drop=True)
pos_df

,textID,text,sentiment,selected_text,wcnt
0,0142108215,Thank you so much phaoloo !!!!,positive,Thank you,2
1,9a2c6ae21c,Ohh i forgot to tell you last night that when ...,positive,amazing,1
2,3f0f5891ab,love your books,positive,love your books,3
3,ac4f58fce3,@_NyLana_ An! Happy Mother`s Day!,positive,Happy Mother`s Day!,3
4,683149f529,well maybe someone will care,positive,well maybe someone will care,5
...,...,...,...,...,...
685,0010bcc0e2,"watching a film, recovering from my operation",positive,recovering from my operation,4
686,18ee7c255c,oh I know. and I aprreciate it it can be my b...,positive,aprreciate,1
687,d65a8658b0,I just got through doing my work now I can tak...,positive,its great,2
688,136b1926c9,Cool. That`d be fantastic!,positive,Cool. That`d be fantastic!,4


In [195]:
neg_df = df[df.sentiment == 'negative'].reset_index(drop=True)
# neg_df = neg_df[neg_df.wcnt > 1].reset_index(drop=True)
neg_df = neg_df.drop_duplicates(['selected_text'], keep='first').reset_index(drop=True)
neg_df

,textID,text,sentiment,selected_text,wcnt
0,fb08563a7b,Midnight ice-cream weather! So **** bored,negative,bored,1
1,32cde6dbc5,in school w. linda doing nothing ;i miss you,negative,miss you,2
2,02073f1c62,I wanna crawl under my desk and take a nap nv...,negative,dirty,1
3,4c2cb5387a,god i cant even catch public transport. swine ...,negative,**** house,2
4,d792349406,she lookd soo freaked out!!poor wee thing,negative,freaked out!!,2
...,...,...,...,...,...
768,8244eb7ae9,"omg, NO ICECREAM",negative,"omg, NO ICECREAM",3
769,fa625455c7,Everyone is working tonight! I`m bored,negative,I`m bored,2
770,8c5e613c41,"Hey, did his wife die? I am really confused. I...",negative,confused.,1
771,e4d58eb70f,IS ANYONE AWAKE?!?!?!?! oh my god i am dying h...,negative,oh my god i am dying here.,7


In [196]:
neu_cnt = int(len(neg_df.textID.values) * 1.29)

In [207]:
neu_df = df[df.sentiment == 'neutral'].reset_index(drop=True)
# neu_df = neu_df.sample(n=neu_cnt, random_state=42).reset_index(drop=True)
neu_df

,textID,text,sentiment,selected_text,wcnt
0,5de3c34293,i am the only arabic girl who`s online every ...,neutral,i am the only arabic girl who`s online every ...,14
1,cf0d831059,starting the video editing of the first spanki...,neutral,starting the video editing of the first spank...,22
2,2d361cdd4d,I have it. Hehehehe u want the torrent?,neutral,I have it. Hehehehe u want the torrent?,8
3,cb4adb2254,I wanna do something tonight after work.... Bu...,neutral,I wanna do something tonight after work.... B...,11
4,1b5dea3fe9,"Early bird gets the worm. And in my case, bir...",neutral,"Early bird gets the worm. And in my case, bir...",15
...,...,...,...,...,...
1422,3e661b33eb,Ill catch you at the very last second,neutral,Ill catch you at the very last second,8
1423,ad970e1d3f,it`s saying that it is the wrong link-it canno...,neutral,it`s saying that it is the wrong link-it cann...,12
1424,eeb047e369,On way back home after a crazy night on the coast,neutral,On way back home after a crazy night on the c...,11
1425,1e1cb8d9a4,let me dream about my spock now goodnight,neutral,let me dream about my spock now goodnight,8


In [208]:
df_concat = pd.concat([pos_df, neg_df, neu_df], axis=0, ignore_index=True)
df_concat

,textID,text,sentiment,selected_text,wcnt
0,0142108215,Thank you so much phaoloo !!!!,positive,Thank you,2
1,9a2c6ae21c,Ohh i forgot to tell you last night that when ...,positive,amazing,1
2,3f0f5891ab,love your books,positive,love your books,3
3,ac4f58fce3,@_NyLana_ An! Happy Mother`s Day!,positive,Happy Mother`s Day!,3
4,683149f529,well maybe someone will care,positive,well maybe someone will care,5
...,...,...,...,...,...
2885,3e661b33eb,Ill catch you at the very last second,neutral,Ill catch you at the very last second,8
2886,ad970e1d3f,it`s saying that it is the wrong link-it canno...,neutral,it`s saying that it is the wrong link-it cann...,12
2887,eeb047e369,On way back home after a crazy night on the coast,neutral,On way back home after a crazy night on the c...,11
2888,1e1cb8d9a4,let me dream about my spock now goodnight,neutral,let me dream about my spock now goodnight,8


In [209]:
df_concat.sentiment.value_counts()

neutral     1427
negative     773
positive     690
Name: sentiment, dtype: int64

In [210]:
df_concat = df_concat.sample(frac=1, random_state=42).reset_index(drop=True)

In [211]:
df_concat = df_concat.drop(['wcnt'], axis=1)
df_concat

,textID,text,sentiment,selected_text
0,31d102fb4c,my darling dd says 'my mum is my best friend'....,positive,love ya moo
1,4e1101e520,Its weird being at the guy`s house without the...,negative,I don`t like it
2,3da51fc31b,"Inorite, humid as well",neutral,"Inorite, humid as well"
3,3dd127f4af,hey hunnie how are u?? I miss talkin to u! Ty...,neutral,hey hunnie how are u?? I miss talkin to u! Ty...
4,ad970e1d3f,it`s saying that it is the wrong link-it canno...,neutral,it`s saying that it is the wrong link-it cann...
...,...,...,...,...
2885,dfe5909132,imma miss dem KObe nd Bron commrcials wen clev...,neutral,imma miss dem KObe nd Bron commrcials wen cle...
2886,cff0e3e256,i feel very ill. i think im dehydrated,negative,i feel very ill.
2887,98cfd1162e,aww...thats a bummer,negative,bummer
2888,87daa62920,4n? ma rog never heard of it esti beat acum?...,negative,u tweet too much


In [212]:
output_path = './dataset/pseudo_filtered.csv'

In [213]:
df_concat.to_csv(output_path, index=False)